# Data_Wrangling_From_Basketball_Dataset

## 1.Data Collection

# 1.1 Query Data From Data Base

In [1]:
from sqlalchemy import create_engine
import pandas as pd
import datetime as dt
engine = create_engine('sqlite:///basketball.sqlite')

In [2]:
# creat query
query ="""
WITH salary AS (
    SELECT nameTeam, namePlayer, AVG(value) AS current_avg_salary
    FROM Player_Salary
    WHERE isOnRoster = 1
    GROUP BY namePlayer)

SELECT *
FROM Player_Attributes AS PA
INNER JOIN salary
ON PA.DISPLAY_FIRST_LAST = salary.namePlayer
"""

In [3]:
# connect to the engine, query data and save it to a data frame
with engine.connect() as con:
    rs = con.execute(query)
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()


In [4]:
print(df.head(10))

        ID FIRST_NAME      LAST_NAME      DISPLAY_FIRST_LAST  \
0   203500     Steven          Adams            Steven Adams   
1  1628389        Bam        Adebayo             Bam Adebayo   
2   200746   LaMarcus       Aldridge       LaMarcus Aldridge   
3  1628960    Grayson          Allen           Grayson Allen   
4  1628386    Jarrett          Allen           Jarrett Allen   
5   203937       Kyle       Anderson           Kyle Anderson   
6   203507    Giannis  Antetokounmpo   Giannis Antetokounmpo   
7  1628961     Kostas  Antetokounmpo    Kostas Antetokounmpo   
8   203648   Thanasis  Antetokounmpo  Thanasis Antetokounmpo   
9     2546    Carmelo        Anthony         Carmelo Anthony   

  DISPLAY_LAST_COMMA_FIRST   DISPLAY_FI_LAST             PLAYER_SLUG  \
0            Adams, Steven          S. Adams            steven-adams   
1             Adebayo, Bam        B. Adebayo             bam-adebayo   
2       Aldridge, LaMarcus       L. Aldridge       lamarcus-aldridge   
3      

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353 entries, 0 to 352
Data columns (total 40 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ID                                353 non-null    object 
 1   FIRST_NAME                        353 non-null    object 
 2   LAST_NAME                         353 non-null    object 
 3   DISPLAY_FIRST_LAST                353 non-null    object 
 4   DISPLAY_LAST_COMMA_FIRST          353 non-null    object 
 5   DISPLAY_FI_LAST                   353 non-null    object 
 6   PLAYER_SLUG                       353 non-null    object 
 7   BIRTHDATE                         353 non-null    object 
 8   SCHOOL                            353 non-null    object 
 9   COUNTRY                           353 non-null    object 
 10  LAST_AFFILIATION                  353 non-null    object 
 11  HEIGHT                            353 non-null    float64
 12  WEIGHT  

In [6]:
# since ALL_STAR_APPEARANCES only have 4 non-null vaules, I am going to drop it
df = df.drop('ALL_STAR_APPEARANCES', axis = 1)

In [7]:
# fill in the null vaule in PIE with average 
df['PIE'].fillna((df['PIE'].mean()), inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353 entries, 0 to 352
Data columns (total 39 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ID                                353 non-null    object 
 1   FIRST_NAME                        353 non-null    object 
 2   LAST_NAME                         353 non-null    object 
 3   DISPLAY_FIRST_LAST                353 non-null    object 
 4   DISPLAY_LAST_COMMA_FIRST          353 non-null    object 
 5   DISPLAY_FI_LAST                   353 non-null    object 
 6   PLAYER_SLUG                       353 non-null    object 
 7   BIRTHDATE                         353 non-null    object 
 8   SCHOOL                            353 non-null    object 
 9   COUNTRY                           353 non-null    object 
 10  LAST_AFFILIATION                  353 non-null    object 
 11  HEIGHT                            353 non-null    float64
 12  WEIGHT  

In [9]:
# drop columns that do not relate the salary
drop_columns = ['ID', 'FIRST_NAME', 'LAST_NAME', 'DISPLAY_LAST_COMMA_FIRST', 'DISPLAY_FI_LAST', 
                'PLAYER_SLUG', 'COUNTRY', 'ROSTERSTATUS', 'GAMES_PLAYED_CURRENT_SEASON_FLAG', 
               'TEAM_ID', 'TEAM_CODE', 'PLAYERCODE', 'DLEAGUE_FLAG', 'NBA_FLAG', 'GAMES_PLAYED_FLAG',
               'nameTeam', 'namePlayer']
df = df.drop(drop_columns, axis = 1)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353 entries, 0 to 352
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   DISPLAY_FIRST_LAST  353 non-null    object 
 1   BIRTHDATE           353 non-null    object 
 2   SCHOOL              353 non-null    object 
 3   LAST_AFFILIATION    353 non-null    object 
 4   HEIGHT              353 non-null    float64
 5   WEIGHT              353 non-null    float64
 6   SEASON_EXP          353 non-null    int64  
 7   JERSEY              353 non-null    object 
 8   POSITION            353 non-null    object 
 9   TEAM_NAME           353 non-null    object 
 10  TEAM_ABBREVIATION   353 non-null    object 
 11  TEAM_CITY           353 non-null    object 
 12  FROM_YEAR           353 non-null    object 
 13  TO_YEAR             353 non-null    object 
 14  DRAFT_YEAR          353 non-null    object 
 15  DRAFT_ROUND         353 non-null    object 
 16  DRAFT_NU

In [14]:
# get age from birthdate
df['BIRTHDATE'] = pd.to_datetime(df['BIRTHDATE'], format="%Y-%m-%dT%H:%M:%S")

In [15]:
type(df.BIRTHDATE[1])

pandas._libs.tslibs.timestamps.Timestamp

In [16]:
# Year difference
today = pd.to_datetime('today')
diff_y = today.year - df['BIRTHDATE'].dt.year
# Haven't had birthday
b_md = df['BIRTHDATE'].apply(lambda x: (x.month,x.day) )
no_birthday = b_md > (today.month,today.day)
df['age'] = diff_y - no_birthday

In [17]:
df.head()

,DISPLAY_FIRST_LAST,BIRTHDATE,SCHOOL,LAST_AFFILIATION,HEIGHT,WEIGHT,SEASON_EXP,JERSEY,POSITION,TEAM_NAME,...,TO_YEAR,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,PTS,AST,REB,PIE,current_avg_salary,age
0,Steven Adams,1993-07-20,Pittsburgh,Pittsburgh/New Zealand,83.0,265.0,7,12,Center,Pelicans,...,2020,2013,1,12,8.1,2.0,9.2,0.093,2.153090e+07,27
1,Bam Adebayo,1997-07-18,Kentucky,Kentucky/USA,81.0,255.0,3,13,Center-Forward,Heat,...,2020,2017,1,14,19.2,5.4,9.5,0.167,2.620383e+07,23
2,LaMarcus Aldridge,1985-07-19,Texas,Texas/USA,83.0,250.0,14,12,Center-Forward,Spurs,...,2020,2006,1,2,13.7,1.7,4.5,0.096,5.549880e+05,35
3,Grayson Allen,1995-10-08,Duke,Duke/USA,76.0,198.0,2,3,Guard,Grizzlies,...,2020,2018,1,21,9.5,2.1,3.1,0.085,4.147610e+06,25
4,Jarrett Allen,1998-04-21,Texas,Texas/USA,83.0,243.0,3,31,Center,Cavaliers,...,2020,2017,1,22,12.9,1.6,9.9,0.147,5.807674e+06,23


In [19]:
# get year played
df.FROM_YEAR = pd.to_numeric(df.FROM_YEAR, errors='coerce')
df.TO_YEAR = pd.to_numeric(df.TO_YEAR, errors='coerce')

In [20]:
df['year_played'] = df.TO_YEAR - df.FROM_YEAR

In [24]:
# drop more columns
drop2 = ['BIRTHDATE', 'FROM_YEAR', 'TO_YEAR']
df = df.drop(drop2, axis = 1)

In [25]:
df.head()

,DISPLAY_FIRST_LAST,SCHOOL,LAST_AFFILIATION,HEIGHT,WEIGHT,SEASON_EXP,JERSEY,POSITION,TEAM_NAME,TEAM_ABBREVIATION,...,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,PTS,AST,REB,PIE,current_avg_salary,age,year_played
0,Steven Adams,Pittsburgh,Pittsburgh/New Zealand,83.0,265.0,7,12,Center,Pelicans,NOP,...,2013,1,12,8.1,2.0,9.2,0.093,2.153090e+07,27,7
1,Bam Adebayo,Kentucky,Kentucky/USA,81.0,255.0,3,13,Center-Forward,Heat,MIA,...,2017,1,14,19.2,5.4,9.5,0.167,2.620383e+07,23,3
2,LaMarcus Aldridge,Texas,Texas/USA,83.0,250.0,14,12,Center-Forward,Spurs,SAS,...,2006,1,2,13.7,1.7,4.5,0.096,5.549880e+05,35,14
3,Grayson Allen,Duke,Duke/USA,76.0,198.0,2,3,Guard,Grizzlies,MEM,...,2018,1,21,9.5,2.1,3.1,0.085,4.147610e+06,25,2
4,Jarrett Allen,Texas,Texas/USA,83.0,243.0,3,31,Center,Cavaliers,CLE,...,2017,1,22,12.9,1.6,9.9,0.147,5.807674e+06,23,3


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353 entries, 0 to 352
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   DISPLAY_FIRST_LAST  353 non-null    object 
 1   SCHOOL              353 non-null    object 
 2   LAST_AFFILIATION    353 non-null    object 
 3   HEIGHT              353 non-null    float64
 4   WEIGHT              353 non-null    float64
 5   SEASON_EXP          353 non-null    int64  
 6   JERSEY              353 non-null    object 
 7   POSITION            353 non-null    object 
 8   TEAM_NAME           353 non-null    object 
 9   TEAM_ABBREVIATION   353 non-null    object 
 10  TEAM_CITY           353 non-null    object 
 11  DRAFT_YEAR          353 non-null    object 
 12  DRAFT_ROUND         353 non-null    object 
 13  DRAFT_NUMBER        353 non-null    object 
 14  PTS                 353 non-null    float64
 15  AST                 353 non-null    float64
 16  REB     